![DME Layers](https://opg.optica.org/getImage.cfm?img=M3cuZnVsbCxib2UtNi00LTExNzItZzAwMw&article=boe-6-4-1172-g003)

-----

![RTA Layers Image](https://journals.plos.org/plosone/article/figure/image?size=large&id=10.1371/journal.pone.0133908.g001) | ![RTA Layers Explanation](https://journals.plos.org/plosone/article/figure/image?size=large&id=10.1371/journal.pone.0133908.t001)
-- | --

![AMD Layers](https://www.ncbi.nlm.nih.gov/pmc/articles/instance/3901571/bin/nihms-508161-f0001.jpg)

----

![AROI Layers](https://ipg.fer.hr/images/50037599/nasa%20baza.png)

In [ ]:
import sys
sys.path.append('..')

In [ ]:
%matplotlib inline 

import os
import re
import copy
from glob import glob
import pathlib
from pathlib import Path
import random
import shutil
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import scipy.io
import imageio
import cv2
from PIL import Image
import idp_utils.data_handling.constants as C
from idp_utils.data_handling.common import (extract_data,
                                            extract_data_aroi,
                                            extract_data_op,
                                            get_dme_valid_idx,
                                            get_amd_valid_idx,
                                            prepare_files)


%cd $C.ROOT_PATH

seed = 6
random.seed(seed)

# 1 Extract Labeled Layeres & Bscans from Data

## 1.1 Original Data

### 1.1.1 RTA

In [ ]:
data, dtype = "RTA", "original"
extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data), "*.mat"), 
                   bscan_key='volumedata', 
                   layermap_key='Observer2',
                   bscan_format='hws',
                   layermap_format='wsl',
                   layer_labels=C.RTA_LABELS,
                   bscan_folder=C.BSCAN_PATTERN.format(data=data),
                   layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                   overwrite=True)

### 1.1.2 DME

In [ ]:
data, dtype = "DME", "original"
extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data), "*.mat"),
                   bscan_key='images',
                   layermap_key='manualLayers1',
                   fluid_key='manualFluid1',
                   bscan_format='hws',
                   layermap_format='lws',
                   layer_labels=C.DME_LABELS,
                   bscan_folder=C.BSCAN_PATTERN.format(data=data),
                   layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                   fluid_folder=C.FLUID_PATTERN.format(data=data),
                   valid_slice_indices_fn=get_dme_valid_idx,
                   overwrite=True)

### 1.1.3 AMD

In [ ]:
# Control data
data, dtype = "AMD", "original"
extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data), 'Control 2/Control 2/*.mat'), 
            bscan_key='images',
            layermap_key='layerMaps',
            bscan_format='hws',
            layermap_format='swl',
            layer_labels=[1, 3, 5], #C.AMD_LABELS
            bscan_folder=C.BSCAN_PATTERN.format(data=data),
            layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
            valid_slice_indices_fn=get_amd_valid_idx,
            overwrite=False,
            save_extension='png')

In [ ]:
# AMD data
data, dtype = "AMD", "original"
extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'AMD 2/AMD 2/*.mat'), 
            bscan_key='images',
            layermap_key='layerMaps',
            bscan_format='hws',
            layermap_format='swl',
            layer_labels=[1, 3, 5], #C.AMD_LABELS
            bscan_folder=C.BSCAN_PATTERN.format(data=data),
            layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
            valid_slice_indices_fn=get_amd_valid_idx,
            overwrite=False,
            save_extension='png')

In [ ]:
# !rm -rf data/extract/bscans/AMD
# !rm -rf data/extract/layers/AMD

### 1.1.4 AROI

In [ ]:
# from PIL import Image
# m = 'data/raw/AROI/24 patient/patient10/mask/number/patient10_raw0040.png'
# img = np.asarray(Image.open(m))
# np.unique(img)
## output:
# array([0, 1, 2, 3, 4, 6], dtype=uint8)

In [ ]:
data, dtype = "AROI", "original"
extract_data_aroi(raw_data_folder=C.RAW_DATA_PATTERN.format(data=data),
                        bscan_folder=C.BSCAN_PATTERN.format(data=data),
                        layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                        fluid_folder=C.FLUID_PATTERN.format(data=data),
                        dtype=dtype,
                        fluid_labels=C.FLUID_LABELS,
                        layer_labels=C.AROI_LABELS)

#### 1.1.4.1 AROI Hetero

We create another AROI dataset that use a totally different set of notations than OP. This is for a later experiment.

In [ ]:
data, dtype = "AROI", "hetero"

hetero_aroi_layer_labels = [22, 57, 174, 190] # was [19, 57, 171, 190]

extract_data_aroi(raw_data_folder=C.RAW_DATA_PATTERN.format(data=data),
                        bscan_folder=C.BSCAN_PATTERN.format(data=data),
                        layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                        fluid_folder=C.FLUID_PATTERN.format(data=data),
                        dtype=dtype,
                        fluid_labels=C.FLUID_LABELS,
                        layer_labels=hetero_aroi_layer_labels)

### 1.1.5 OP

In [ ]:
data, dtype = "OP", "original"
name = "original"

extract_data_op(raw_data_folder=C.RAW_DATA_PATTERN.format(data=data),
                bscan_folder=C.BSCAN_PATTERN.format(data=data),
                layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                layer_labels=C.OP_LABELS,
                instrument_labels=C.INSTRUMENT_LABELS,
                save_extension='png')

In [ ]:
data, dtype = "ioct", "original"
name = "original"

extract_data_op(raw_data_folder=C.RAW_DATA_PATTERN.format(data="OP"),
                bscan_folder=C.BSCAN_PATTERN.format(data=data),
                layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                layer_labels=[1,2],
                instrument_labels=[3,4],
                save_extension='png')

In [ ]:
import os
import re
import copy
from glob import glob
import pathlib
from pathlib import Path
import random
import shutil
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import scipy.io
import imageio
import cv2
import torch
from PIL import Image
from idp_utils.data_handling.constants import (INSTRUMENT_LABELS, 
                                               BSCAN_PATTERN,
                                               LAYER_PATTERN,
                                               FLUID_PATTERN,
                                               FLUID_LABELS,
                                               AROI_LABEL_DICT)

In [ ]:
def extract_data_op(raw_data_folder, bscan_folder, layer_folder, layer_labels, instrument_labels=None, save_extension='png'):

    raw_layer_labels = [1, 3]
    raw_instrument_labels = [2, 4]

    if not Path(bscan_folder).exists():
        Path(bscan_folder).mkdir(parents=True)
        print(f"Created folder {bscan_folder}")

    if not Path(layer_folder).exists():
        Path(layer_folder).mkdir(parents=True)
        print(f"Created folder {layer_folder}")

    parts = glob(os.path.join(raw_data_folder, "*"))
    for part in parts:
        print(f"{part.split('/')[-1]} Started")
        folders = glob(os.path.join(part, "*"))
        for folder in tqdm(folders):
            folder_name = folder.split('/')[-1].split('.')[0]
            bscans = glob(os.path.join(folder, "[0-9]*.bmp"))
            layers = glob(os.path.join(folder, "segmentation", "[0-9]*.bmp"))
            assert len(bscans) !=0 and len(layers) != 0 and len(bscans) == len(layers)

            for layer in layers:
                layer_name = folder_name + "-" + layer.split('/')[-1].split('.')[0] + '.' + save_extension
                layer_arr = np.asarray(Image.open(layer))
                # for i in range(2):
                #     layer_arr[layer_arr == raw_layer_labels[i]] = layer_labels[i]
                # if instrument_labels:
                #     assert len(instrument_labels) >= len(raw_instrument_labels), \
                #         f"instrument_labels ({len(instrument_labels)}) is not enough (expect {len(raw_instrument_labels)})"
                #     for i in range(2):
                #         layer_arr[layer_arr == raw_instrument_labels[i]] = instrument_labels[i]
                # else:
                #     layer_arr[layer_arr == raw_instrument_labels[i]] = INSTRUMENT_LABELS[i]
                layer_img = Image.fromarray(layer_arr)
                layer_img.save(os.path.join(layer_folder, layer_name))
            
            for bscan in bscans:
                bscan_name = folder_name + "-" + bscan.split('/')[-1].split('.')[0] + '.' + save_extension
                bscan_arr = np.asarray(Image.open(bscan))
                bscan_img = Image.fromarray(bscan_arr)
                bscan_img.save(os.path.join(bscan_folder, bscan_name))

In [ ]:
data, dtype = "iOCT", "original"
name = "original"

extract_data_op(raw_data_folder=C.RAW_DATA_PATTERN.format(data="OP"),
                bscan_folder=C.BSCAN_PATTERN.format(data=data),
                layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                layer_labels=[1,2],
                instrument_labels=[3,4],
                save_extension='png')

In [ ]:
import imageio
import numpy as np
from glob import glob

test_layers = glob(C.LAYER_PATTERN.format(data=data, dtype=dtype) + '/*.png')
print(len(test_layers))
layer = test_layers[0]

In [ ]:
im = imageio.imread(layer)
np.unique(im)

## 1.2 Reduced-Layer Data

### 1.2.1 RTA

In [ ]:
# data = "RTA"
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject[78]*.mat'), 
#                    bscan_key='volumedata', 
#                    layermap_key='Observer2',
#                    bscan_format='hws',
#                    layermap_format='wsl',
#                    layer_labels=C.RTA_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce1'),
#                    remove_from=range(2,8), # 6 out of 8
#                    n_remove=1)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject[56]*.mat'), 
#                    bscan_key='volumedata', 
#                    layermap_key='Observer2',
#                    bscan_format='hws',
#                    layermap_format='wsl',
#                    layer_labels=C.RTA_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce2'),
#                    remove_from=range(2,8), # 6 out of 8
#                    n_remove=2)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject[34]*.mat'), 
#                    bscan_key='volumedata', 
#                    layermap_key='Observer2',
#                    bscan_format='hws',
#                    layermap_format='wsl',
#                    layer_labels=C.RTA_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce3'),
#                    remove_from=range(2,8), # 6 out of 8
#                    n_remove=3)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject[12]*.mat'), 
#                    bscan_key='volumedata', 
#                    layermap_key='Observer2',
#                    bscan_format='hws',
#                    layermap_format='wsl',
#                    layer_labels=C.RTA_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce4'),
#                    remove_from=range(2,8), # 6 out of 8
#                    n_remove=4)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject[12]*.mat'), 
#                    bscan_key='volumedata', 
#                    layermap_key='Observer2',
#                    bscan_format='hws',
#                    layermap_format='wsl',
#                    layer_labels=C.RTA_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce5'),
#                    remove_from=range(2,8), # 6 out of 8
#                    n_remove=5)

### 1.2.2 DME

In [ ]:
# data = "DME"
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject_0[89].mat'),
#                    bscan_key='images',
#                    layermap_key='manualLayers1',
#                    fluid_key='manualFluid1',
#                    bscan_format='hws',
#                    layermap_format='lws',
#                    layer_labels=C.DME_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce1'),
#                    fluid_folder=C.FLUID_PATTERN.format(data=data),
#                    valid_slice_indices_fn=get_dme_valid_idx,
#                    remove_from=range(2,8), # DME also has 8 layers, we choose 6 out of 8
#                    n_remove=1)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject_0[67].mat'),
#                    bscan_key='images',
#                    layermap_key='manualLayers1',
#                    fluid_key='manualFluid1',
#                    bscan_format='hws',
#                    layermap_format='lws',
#                    layer_labels=C.DME_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce2'),
#                    fluid_folder=C.FLUID_PATTERN.format(data=data),
#                    valid_slice_indices_fn=get_dme_valid_idx,
#                    remove_from=range(2,8), # DME also has 8 layers, we choose 6 out of 8
#                    n_remove=2)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject_0[45].mat'),
#                    bscan_key='images',
#                    layermap_key='manualLayers1',
#                    fluid_key='manualFluid1',
#                    bscan_format='hws',
#                    layermap_format='lws',
#                    layer_labels=C.DME_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce3'),
#                    fluid_folder=C.FLUID_PATTERN.format(data=data),
#                    valid_slice_indices_fn=get_dme_valid_idx,
#                    remove_from=range(2,8), # DME also has 8 layers, we choose 6 out of 8
#                    n_remove=3)
# extract_data(file_pattern=os.path.join(C.RAW_DATA_PATTERN.format(data=data),'Subject_0[23].mat'),
#                    bscan_key='images',
#                    layermap_key='manualLayers1',
#                    fluid_key='manualFluid1',
#                    bscan_format='hws',
#                    layermap_format='lws',
#                    layer_labels=C.DME_LABELS,
#                    bscan_folder=C.BSCAN_PATTERN.format(data=data),
#                    layer_folder=C.LAYER_PATTERN.format(data=data, dtype='reduce4'),
#                    fluid_folder=C.FLUID_PATTERN.format(data=data),
#                    valid_slice_indices_fn=get_dme_valid_idx,
#                    remove_from=range(2,8), # DME also has 8 layers, we choose 6 out of 8
#                    n_remove=4)

### 1.2.3 AMD

There are only 3 layers within it. Let it go.

### 1.2.4 AROI

In [ ]:
data = 'AROI'
dtype = 'reduce1'
extract_data_aroi(raw_data_folder=C.RAW_DATA_PATTERN.format(data=data),
                 bscan_folder=C.BSCAN_PATTERN.format(data=data),
                 layer_folder=C.LAYER_PATTERN.format(data=data, dtype=dtype),
                 fluid_folder=C.FLUID_PATTERN.format(data=data),
                 dtype=dtype,
                 fluid_labels=C.FLUID_LABELS,
                 layer_labels=C.AROI_LABELS,
                 remove_from=range(1,3),
                 n_remove=1)

# 2 Split Data into train, val and test

## 2.1 Original Data

### 2.1.1 RTA

In [ ]:
data, dtype = "RTA", "original"
name = "original"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=False,
              dtype=dtype  )

### 2.1.2 DME

Just a backup for original code
```python
bscan_files = glob(C.DME_BSCAN_ORIGINAL_FOLDER + '*.jpg')
assert len(bscan_files) != 0

train_files, test_files, val_files = split_files(bscan_files, 0.8, 0.1)
assert len(train_files) != 0, f"train_files is empty, data may have already been moved"

splited_files = {
    'train': train_files,
    'test': test_files,
    'val': val_files
}
move_files(splited_files, C.DME_LAYER_BSCAN_ORIGINAL_ROOT, C.DME_SPLIT_ORIGINAL_FOLDER)
```

In [ ]:
data, dtype = "DME", "original"
name = "original"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=True,
              dtype=dtype)

### 2.1.3 AMD

In [ ]:
data, dtype = "AMD", "original"
name = "original"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=False,
              dtype=dtype,
              extension='png')

Copy files to another project:

In [ ]:
%%bash

for f in /home/extra/micheal/IDP/data/splits/AMD/original/bscans/train/*; do ln "$f" /home/extra/micheal/pixel2style2pixel/data/amd/bscans/train/; done
ln /home/extra/micheal/IDP/data/splits/AMD/original/bscans/val/* /home/extra/micheal/pixel2style2pixel/data/amd/bscans/val/
ln /home/extra/micheal/IDP/data/splits/AMD/original/bscans/test/* /home/extra/micheal/pixel2style2pixel/data/amd/bscans/test/
for f in ln /home/extra/micheal/IDP/data/splits/AMD/original/labels/train/*; do ln "$f" /home/extra/micheal/pixel2style2pixel/data/amd/labels/train/; done
ln /home/extra/micheal/IDP/data/splits/AMD/original/labels/val/* /home/extra/micheal/pixel2style2pixel/data/amd/labels/val/
ln /home/extra/micheal/IDP/data/splits/AMD/original/labels/test/* /home/extra/micheal/pixel2style2pixel/data/amd/labels/test/

### 2.1.4 AROI

In [ ]:
data, dtype = "AROI", "original"
name = "original"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=True,
              dtype=dtype,
              extension='png')

#### 2.1.4.1 AROI Hetero

In [ ]:
data, dtype = "AROI", "hetero"
name = "hetero"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=True,
              dtype=dtype,
              extension='png')

### 2.1.5 OP

In [ ]:
data, dtype = "iOCT", "original"
name = "original"

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data="ioct", name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=False,
              dtype=dtype,
              extension='png')

## 2.2 Reduced-Layer Data

### 2.2.1 RTA

just another backup:
```python
bscan_files = glob(C.RTA_BSCAN_REDUCED_FOLDER+'*.jpg')
print(len(bscan_files))

train_files, test_files, val_files = split_files(bscan_files, 0.8, 0.1)
assert len(train_files) != 0, f"train_files is empty, data may have already been moved"

splited_files = {
    'train': train_files,
    'test': test_files,
    'val': val_files
}
move_files(splited_files, C.RTA_LAYER_BSCAN_REDUCED_ROOT, C.RTA_SPLIT_REDUCED_FOLDER)
```

In [ ]:
# data = "RTA"
# name = "reduce_merge"

# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=False,
#               dtype="reduce1", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=False,
#               dtype="reduce2", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=False,
#               dtype="reduce3", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=False,
#               dtype="reduce4", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=False,
#               dtype="reduce5", 
#               merge_original=True)

### 2.2.2 DME

In [ ]:
# data = "DME"
# name = "reduce_merge"

# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=True,
#               dtype="reduce1", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=True,
#               dtype="reduce2", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=True,
#               dtype="reduce3", 
#               merge_original=True)
# prepare_files(data=data, 
#               dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
#               train_ratio=0.8, 
#               test_ratio=0.1,
#               with_fluids=True,
#               dtype="reduce4", 
#               merge_original=True)

### 2.2.3 AROI

In [ ]:
data, dtype = "AROI", "reduce1"
name = "reduce1_merge"
merge_original = True

prepare_files(data=data, 
              dst_folder=C.SPLIT_PATTERN.format(data=data, name=name), 
              train_ratio=0.8, 
              test_ratio=0.1,
              with_fluids=True,
              dtype=dtype, 
              merge_original=merge_original,
              extension='png')

# 3 Create pix2pix Compatible Datasets

## 3.1 Original Data

### 3.1.1 RTA

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='RTA', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='RTA', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='RTA', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A $label_folder --fold_B $bscan_folder --fold_AB $dataset_folder

### 3.1.2 DME

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='DME', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='DME', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='DME', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $label_folder \
    --fold_B $bscan_folder \
    --fold_AB $dataset_folder

### 3.1.3 AMD

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='AMD', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='AMD', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='AMD', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A $label_folder --fold_B $bscan_folder --fold_AB $dataset_folder

In [ ]:
# delete generated AMD datasets
!rm $dataset_folder/test/Farsiu_*
!rm $dataset_folder/train/Farsiu_*
!rm $dataset_folder/val/Farsiu_*

### 3.1.4 AROI

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='AROI', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='AROI', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='AROI', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $label_folder \
    --fold_B $bscan_folder \
    --fold_AB $dataset_folder

#### 3.1.4.1 AROI Hetero

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='AROI', name='hetero') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='AROI', name='hetero') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='AROI', name='hetero')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $label_folder \
    --fold_B $bscan_folder \
    --fold_AB $dataset_folder

### 3.1.5 OP

In [ ]:
label_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/labels'
bscan_folder = C.SPLIT_PATTERN.format(data='OP', name='original') + '/bscans'
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $label_folder \
    --fold_B $bscan_folder \
    --fold_AB $dataset_folder

## 3.2 Reduced-Layer Data

### 3.2.1 RTA

In [ ]:
# data, name = "RTA", "reduce_merge"
# bscan_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "bscans")
# label_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "labels")
# dataset_folder = C.DATASET_PATTERN.format(data=data, name=name)

# !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
#     --fold_A $label_folder \
#     --fold_B $bscan_folder \
#     --fold_AB $dataset_folder

### 3.2.2 DME

In [ ]:
# data, name = "DME", "reduce_merge"
# bscan_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "bscans")
# label_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "labels")
# dataset_folder = C.DATASET_PATTERN.format(data=data, name=name)

# !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
#     --fold_A $label_folder \
#     --fold_B $bscan_folder \
#     --fold_AB $dataset_folder

### 3.2.3 AROI

In [ ]:
data, name = "AROI", "reduce1_merge"

bscan_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "bscans")
label_folder = os.path.join(C.SPLIT_PATTERN.format(data=data, name=name), "labels")
dataset_folder = C.DATASET_PATTERN.format(data=data, name=name)

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A "$label_folder" \
    --fold_B "$bscan_folder" \
    --fold_AB "$dataset_folder" 